In [9]:
import os
import pandas as pd
from bs4 import BeautifulSoup
from charset_normalizer import from_path  # エンコーディング自動検出用


In [10]:
# HTMLファイルが格納されているディレクトリのパス
directory = "www.geocities.co.jp"

In [14]:
def extract_text_from_html(file_path):
    """
    HTMLファイルからテキストを抽出する関数
    """
    try:
        # charset-normalizerを使ってファイルを自動的に読み込む
        result = from_path(file_path).best()
        content = str(result)  # decodeの代わりにstr()で文字列に変換
        soup = BeautifulSoup(content, 'html.parser')
        return soup.get_text(separator='\n')
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None  # エラーが発生した場合、空のテキストを返す

In [18]:
def extract_text_recursive_to_df(directory, max_pages=100):
    """
    指定されたディレクトリを再帰的に検索し、HTMLファイルからテキストを抽出し、
    ディレクトリ構造をキーとしてDataFrameに格納する関数
    """
    data = []
    page_count = 0
    
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".html") or file.endswith(".htm"):
                file_path = os.path.join(root, file)
                text = "".join(extract_text_from_html(file_path))
                if text is None: continue
                # ディレクトリ構造と抽出したテキストをDataFrameに格納するためリストに追加
                data.append({"directory": root + "_".join(dirs), "file": file, "text": text[:500]})  # 500文字のみ格納
                
                page_count += 1
                if page_count >= max_pages:
                    print(f"Processed {max_pages} pages. Stopping...")
                    return pd.DataFrame(data)  # DataFrameに変換して返す
    
    return pd.DataFrame(data)

In [19]:
# 実行
df = extract_text_recursive_to_df(directory)

# データフレームを表示
print(df.head())

Processed 100 pages. Stopping...
                                           directory           file  \
0  www.geocities.co.jp/Playtown-Domino/6517gagaga...   geobook.html   
1  www.geocities.co.jp/Playtown-Domino/6517gagaga...  seltutei.html   
2  www.geocities.co.jp/Playtown-Domino/6517gagaga...    ninjya.html   
3  www.geocities.co.jp/Playtown-Domino/6517gagaga...     index.html   
4  www.geocities.co.jp/Playtown-Domino/6517gagaga...  geodiary.html   

                                                text  
0  \n\n\n\nGGGゲストブック\n\n\n\n\n\n\n\n\n僕のゲストブックです、...  
1  \n\n\n\n\n\n\n\n\n\n\n\n\n\n特捜勇者\n\n\n\n\n\n\n...  
2  \n\n\n\n\n\n\n\n\n\n\n\n\n\n第1話\n\n\n\n\n\n\n\...  
3  \n\n\n\n\n\n\n\nＧＧＧ奈良支部ドラえもん課\n\n\n\n\n\n\n\n\...  
4  \n\n\n\n我王神地のプラモデル日記\n\n\n\n\n\n\n\n\n\n\n\n\n...  
